<a href="https://colab.research.google.com/github/J3rmed/ai4eng_2025_2/blob/main/02-preprocesado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 02 - Preprocesado de Datos

En este notebook, se llevará a cabo el proceso de limpieza y preprocesado del conjunto de datos `train.csv`. El objetivo es transformar los datos crudos en un formato numérico, limpio y consistente, adecuado para ser utilizado en el entrenamiento de modelos de Machine Learning.

Las principales tareas a realizar son:

*   **Limpieza inicial:** Eliminación de columnas irrelevantes o duplicadas.
*   **Imputación de datos faltantes:** Manejo de los valores `NaN` (nulos) presentes en el dataset.
*   **Conversión de variables categóricas:** Transformación de columnas de tipo texto a un formato numérico, utilizando diferentes estrategias según la naturaleza de la variable (ordinal, binaria o nominal).
*   **Codificación de la variable objetivo:** Conversión de la etiqueta de rendimiento a un formato numérico.


## Importante: cargar kaggle token

In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = '.'
!chmod 600 ./kaggle.json
!kaggle competitions download -c udea-ai-4-eng-20252-pruebas-saber-pro-colombia

udea-ai-4-eng-20252-pruebas-saber-pro-colombia.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
!unzip udea*.zip > /dev/null

replace submission_example.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
!wc *.csv

   296787    296787   4716673 submission_example.csv
   296787   4565553  59185238 test.csv
   692501  10666231 143732437 train.csv
  1286075  15528571 207634348 total


In [ ]:
import pandas as pd
import numpy as np

z = pd.read_csv("train.csv")
print ("shape of loaded dataframe", z.shape)

shape of loaded dataframe (692500, 21)


In [ ]:
z.head()

,ID,PERIODO_ACADEMICO,E_PRGM_ACADEMICO,E_PRGM_DEPARTAMENTO,E_VALORMATRICULAUNIVERSIDAD,E_HORASSEMANATRABAJA,F_ESTRATOVIVIENDA,F_TIENEINTERNET,F_EDUCACIONPADRE,F_TIENELAVADORA,...,E_PRIVADO_LIBERTAD,E_PAGOMATRICULAPROPIO,F_TIENECOMPUTADOR,F_TIENEINTERNET.1,F_EDUCACIONMADRE,RENDIMIENTO_GLOBAL,INDICADOR_1,INDICADOR_2,INDICADOR_3,INDICADOR_4
0,904256,20212,ENFERMERIA,BOGOTÁ,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica incompleta,Si,...,N,No,Si,Si,Postgrado,medio-alto,0.322,0.208,0.310,0.267
1,645256,20212,DERECHO,ATLANTICO,Entre 2.5 millones y menos de 4 millones,0,Estrato 3,No,Técnica o tecnológica completa,Si,...,N,No,Si,No,Técnica o tecnológica incompleta,bajo,0.311,0.215,0.292,0.264
2,308367,20203,MERCADEO Y PUBLICIDAD,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Si,...,N,No,No,Si,Secundaria (Bachillerato) completa,bajo,0.297,0.214,0.305,0.264
3,470353,20195,ADMINISTRACION DE EMPRESAS,SANTANDER,Entre 4 millones y menos de 5.5 millones,0,Estrato 4,Si,No sabe,Si,...,N,No,Si,Si,Secundaria (Bachillerato) completa,alto,0.485,0.172,0.252,0.190
4,989032,20212,PSICOLOGIA,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 3,Si,Primaria completa,Si,...,N,No,Si,Si,Primaria completa,medio-bajo,0.316,0.232,0.285,0.294


In [ ]:
z.isna().sum()

,0
ID,0
PERIODO_ACADEMICO,0
E_PRGM_ACADEMICO,0
E_PRGM_DEPARTAMENTO,0
E_VALORMATRICULAUNIVERSIDAD,6287
E_HORASSEMANATRABAJA,30857
F_ESTRATOVIVIENDA,32137
F_TIENEINTERNET,26629
F_EDUCACIONPADRE,23178
F_TIENELAVADORA,39773


In [ ]:
z.dtypes

,0
ID,int64
PERIODO_ACADEMICO,int64
E_PRGM_ACADEMICO,object
E_PRGM_DEPARTAMENTO,object
E_VALORMATRICULAUNIVERSIDAD,object
E_HORASSEMANATRABAJA,object
F_ESTRATOVIVIENDA,object
F_TIENEINTERNET,object
F_EDUCACIONPADRE,object
F_TIENELAVADORA,object



Antes de transformar los datos, debemos eliminar columnas que no aportan información o que están repetidas.

## **Columna: F_TIENEINTERNET.1**

 Esta columna parece ser un duplicado exacto de `F_TIENEINTERNET`. Tener información redundante no ayuda al modelo. Verificamos que si es así, y la eliminamos.


In [ ]:
df_procesado = z.copy()

# Verificamos si las columnas de internet son idénticas.
if 'F_TIENEINTERNET.1' in df_procesado.columns and 'F_TIENEINTERNET' in df_procesado.columns:
    son_iguales = df_procesado[['F_TIENEINTERNET', 'F_TIENEINTERNET.1']].dropna()
    verificacion = (son_iguales['F_TIENEINTERNET'] == son_iguales['F_TIENEINTERNET.1']).all()

    if verificacion:
        df_procesado = df_procesado.drop('F_TIENEINTERNET.1', axis=1)

## **Columna: E_PRIVADO_LIBERTAD**

Es una variable cualitativa binaria (`S` o `N`). Al inspeccionar sus valores, vemos que tiene una distribución extremadamente desbalanceada (todos o casi todos los valores son 'N'). Una columna con muy poca o ninguna variabilidad no tiene poder predictivo.

**Decisión:** Eliminar la columna por su baja varianza.

In [ ]:
print("Valores en la columna 'E_PRIVADO_LIBERTAD':")
print(df_procesado['E_PRIVADO_LIBERTAD'].value_counts())

df_procesado = df_procesado.drop('E_PRIVADO_LIBERTAD', axis=1)

Valores en la columna 'E_PRIVADO_LIBERTAD':
E_PRIVADO_LIBERTAD
N    692466
S        34
Name: count, dtype: int64


In [ ]:
df_procesado.head()

,ID,PERIODO_ACADEMICO,E_PRGM_ACADEMICO,E_PRGM_DEPARTAMENTO,E_VALORMATRICULAUNIVERSIDAD,E_HORASSEMANATRABAJA,F_ESTRATOVIVIENDA,F_TIENEINTERNET,F_EDUCACIONPADRE,F_TIENELAVADORA,F_TIENEAUTOMOVIL,E_PAGOMATRICULAPROPIO,F_TIENECOMPUTADOR,F_EDUCACIONMADRE,RENDIMIENTO_GLOBAL,INDICADOR_1,INDICADOR_2,INDICADOR_3,INDICADOR_4
0,904256,20212,ENFERMERIA,BOGOTÁ,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica incompleta,Si,Si,No,Si,Postgrado,medio-alto,0.322,0.208,0.310,0.267
1,645256,20212,DERECHO,ATLANTICO,Entre 2.5 millones y menos de 4 millones,0,Estrato 3,No,Técnica o tecnológica completa,Si,No,No,Si,Técnica o tecnológica incompleta,bajo,0.311,0.215,0.292,0.264
2,308367,20203,MERCADEO Y PUBLICIDAD,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Si,No,No,No,Secundaria (Bachillerato) completa,bajo,0.297,0.214,0.305,0.264
3,470353,20195,ADMINISTRACION DE EMPRESAS,SANTANDER,Entre 4 millones y menos de 5.5 millones,0,Estrato 4,Si,No sabe,Si,No,No,Si,Secundaria (Bachillerato) completa,alto,0.485,0.172,0.252,0.190
4,989032,20212,PSICOLOGIA,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 3,Si,Primaria completa,Si,Si,No,Si,Primaria completa,medio-bajo,0.316,0.232,0.285,0.294


# **Manejo datos faltantes**

Varias columnas tienen valores faltantes (`NaN`). Para no perder estas filas de datos, debemos rellenar esos vacíos.

**Opciones para los NAN:**

*   **Eliminarlos:** No es ideal, perderíamos muchos datos.
*   **Poner un valor aleatorio:** Puede introducir ruido.
*   **Moda:** Rellenar con el valor más frecuente. Es una estrategia muy buena ya que asigna el valor más probable.

**Decisión:** Usar la moda para rellenar los `NaN` en las columnas.

In [ ]:
# Identificamos las columnas de tipo 'object' (categóricas) con valores nulos.
columnas_categoricas_nulas = [col for col in df_procesado.columns if df_procesado[col].dtype == 'object' and df_procesado[col].isna().any()]

# Rellenamos los NaN con la moda de cada columna.
for col in columnas_categoricas_nulas:
    moda = df_procesado[col].mode()[0]
    df_procesado[col] = df_procesado[col].fillna(moda)

In [ ]:
print(z.isna().sum())

ID                                 0
PERIODO_ACADEMICO                  0
E_PRGM_ACADEMICO                   0
E_PRGM_DEPARTAMENTO                0
E_VALORMATRICULAUNIVERSIDAD     6287
E_HORASSEMANATRABAJA           30857
F_ESTRATOVIVIENDA              32137
F_TIENEINTERNET                26629
F_EDUCACIONPADRE               23178
F_TIENELAVADORA                39773
F_TIENEAUTOMOVIL               43623
E_PRIVADO_LIBERTAD                 0
E_PAGOMATRICULAPROPIO           6498
F_TIENECOMPUTADOR              38103
F_TIENEINTERNET.1              26629
F_EDUCACIONMADRE               23664
RENDIMIENTO_GLOBAL                 0
INDICADOR_1                        0
INDICADOR_2                        0
INDICADOR_3                        0
INDICADOR_4                        0
dtype: int64


In [ ]:
print(df_procesado.isna().sum())

ID                             0
PERIODO_ACADEMICO              0
E_PRGM_ACADEMICO               0
E_PRGM_DEPARTAMENTO            0
E_VALORMATRICULAUNIVERSIDAD    0
E_HORASSEMANATRABAJA           0
F_ESTRATOVIVIENDA              0
F_TIENEINTERNET                0
F_EDUCACIONPADRE               0
F_TIENELAVADORA                0
F_TIENEAUTOMOVIL               0
E_PAGOMATRICULAPROPIO          0
F_TIENECOMPUTADOR              0
F_EDUCACIONMADRE               0
RENDIMIENTO_GLOBAL             0
INDICADOR_1                    0
INDICADOR_2                    0
INDICADOR_3                    0
INDICADOR_4                    0
dtype: int64


## **Columna: PERIODO_ACADEMICO**

Esta columna combina año y semestre. Podríamos separarla en dos columnas año y semestre para que el modelo pueda capturar tendencias anuales y patrones semestrales de forma independiente.


In [ ]:
# Convertimos la columna a texto para poder dividirla.
df_procesado['PERIODO_ACADEMICO'] = df_procesado['PERIODO_ACADEMICO'].astype(str)

# Creamos la columna 'AÑO' con los primeros 4 caracteres.
df_procesado['AÑO'] = df_procesado['PERIODO_ACADEMICO'].str[:4].astype(int)

# Creamos la columna 'SEMESTRE' con el último caracter.
df_procesado['SEMESTRE'] = df_procesado['PERIODO_ACADEMICO'].str[4:].astype(int)

# Eliminamos la columna original.
df_procesado = df_procesado.drop('PERIODO_ACADEMICO', axis=1)


In [ ]:
df_procesado[['ID','AÑO', 'SEMESTRE',]].head()

,ID,AÑO,SEMESTRE
0,904256,2021,2
1,645256,2021,2
2,308367,2020,3
3,470353,2019,5
4,989032,2021,2


In [ ]:
z[['ID','PERIODO_ACADEMICO']].head()

,ID,PERIODO_ACADEMICO
0,904256,20212
1,645256,20212
2,308367,20203
3,470353,20195
4,989032,20212


# **Columnas a Formato Numérico**

Convertimos todas las columnas de texto restantes a números.

## **Columnas: Variables Binarias (Si/No)**

## **Columna: F_TIENEINTERNET**
Posibles valores: "No": 0, "Si": 1.

In [ ]:
df_procesado['F_TIENEINTERNET'] = (df_procesado['F_TIENEINTERNET'] == 'Si').astype(int)

In [ ]:
df_procesado[['ID','F_TIENEINTERNET',]].head(10)

,ID,F_TIENEINTERNET
0,904256,1
1,645256,0
2,308367,1
3,470353,1
4,989032,1
5,659872,1
6,47159,1
7,11829,1
8,257869,1
9,465511,1


In [ ]:
len(df_procesado.F_TIENEINTERNET.unique())

2

## **Columna: E_PAGOMATRICULAPROPIO**
Posibles valores: "No": 0, "Si": 1.

In [ ]:
df_procesado['E_PAGOMATRICULAPROPIO'] = (df_procesado['E_PAGOMATRICULAPROPIO'] == 'Si').astype(int)

In [ ]:
df_procesado[['ID','E_PAGOMATRICULAPROPIO',]].head(10)

,ID,E_PAGOMATRICULAPROPIO
0,904256,0
1,645256,0
2,308367,0
3,470353,0
4,989032,0
5,659872,0
6,47159,1
7,11829,1
8,257869,1
9,465511,1


In [ ]:
len(df_procesado.E_PAGOMATRICULAPROPIO.unique())

2

## **Columna: F_TIENELAVADORA**
Posibles valores: "No": 0, "Si": 1.

In [ ]:
df_procesado['F_TIENELAVADORA'] = (df_procesado['F_TIENELAVADORA'] == 'Si').astype(int)

In [ ]:
df_procesado[['ID','F_TIENELAVADORA',]].head(10)

,ID,F_TIENELAVADORA
0,904256,1
1,645256,1
2,308367,1
3,470353,1
4,989032,1
5,659872,1
6,47159,1
7,11829,1
8,257869,1
9,465511,1


In [ ]:
len(df_procesado.F_TIENELAVADORA.unique())

2

## **Columna: F_TIENEAUTOMOVIL**
Posibles valores: "No": 0, "Si": 1.

In [ ]:
df_procesado['F_TIENEAUTOMOVIL'] = (df_procesado['F_TIENEAUTOMOVIL'] == 'Si').astype(int)

In [ ]:
df_procesado[['ID','F_TIENEAUTOMOVIL',]].head(10)

,ID,F_TIENEAUTOMOVIL
0,904256,1
1,645256,0
2,308367,0
3,470353,0
4,989032,1
5,659872,1
6,47159,1
7,11829,0
8,257869,1
9,465511,1


In [ ]:
len(df_procesado.F_TIENEAUTOMOVIL.unique())

2

## **Columna: F_TIENECOMPUTADOR**
Posibles valores: "No": 0, "Si": 1.

In [ ]:
df_procesado['F_TIENECOMPUTADOR'] = (df_procesado['F_TIENECOMPUTADOR'] == 'Si').astype(int)

In [ ]:
df_procesado[['ID','F_TIENECOMPUTADOR',]].head(10)

,ID,F_TIENECOMPUTADOR
0,904256,1
1,645256,1
2,308367,0
3,470353,1
4,989032,1
5,659872,1
6,47159,1
7,11829,0
8,257869,1
9,465511,1


In [ ]:
len(df_procesado.F_TIENECOMPUTADOR.unique())

2

In [ ]:
df_procesado[['ID','AÑO', 'SEMESTRE','F_TIENECOMPUTADOR','F_TIENEAUTOMOVIL','F_TIENELAVADORA','E_PAGOMATRICULAPROPIO','F_TIENEINTERNET']].head(10)

,ID,AÑO,SEMESTRE,F_TIENECOMPUTADOR,F_TIENEAUTOMOVIL,F_TIENELAVADORA,E_PAGOMATRICULAPROPIO,F_TIENEINTERNET
0,904256,2021,2,1,1,1,0,1
1,645256,2021,2,1,0,1,0,0
2,308367,2020,3,0,0,1,0,1
3,470353,2019,5,1,0,1,0,1
4,989032,2021,2,1,1,1,0,1
5,659872,2020,3,1,1,1,0,1
6,47159,2018,3,1,1,1,1,1
7,11829,2018,3,0,0,1,1,1
8,257869,2021,2,1,1,1,1,1
9,465511,2018,3,1,1,1,1,1


# **Columnas: Variables Ordinales (con un orden lógico)**
Las categorías se mapearán a números del 0 al ... para preservar el orden.

## **Columna: E_VALORMATRICULAUNIVERSIDAD**
Las categorías representan rangos de dinero crecientes. Se mapearán a números del 0 al 7.

In [ ]:
df_procesado.E_VALORMATRICULAUNIVERSIDAD.unique()

array(['Entre 5.5 millones y menos de 7 millones',
       'Entre 2.5 millones y menos de 4 millones',
       'Entre 4 millones y menos de 5.5 millones', 'Más de 7 millones',
       'Entre 1 millón y menos de 2.5 millones',
       'Entre 500 mil y menos de 1 millón', 'Menos de 500 mil',
       'No pagó matrícula'], dtype=object)

In [ ]:
mapa_matricula = {
    'No pagó matrícula': 0, 'Menos de 500 mil': 1, 'Entre 500 mil y menos de 1 millón': 2,
    'Entre 1 millón y menos de 2.5 millones': 3, 'Entre 2.5 millones y menos de 4 millones': 4,
    'Entre 4 millones y menos de 5.5 millones': 5, 'Entre 5.5 millones y menos de 7 millones': 6,
    'Más de 7 millones': 7
}
df_procesado['E_VALORMATRICULAUNIVERSIDAD'] = df_procesado['E_VALORMATRICULAUNIVERSIDAD'].map(mapa_matricula)

In [ ]:
df_procesado.E_VALORMATRICULAUNIVERSIDAD.unique()

array([6, 4, 5, 7, 3, 2, 1, 0])

In [ ]:
df_procesado[['ID','E_VALORMATRICULAUNIVERSIDAD',]].head(10)

,ID,E_VALORMATRICULAUNIVERSIDAD
0,904256,6
1,645256,4
2,308367,4
3,470353,5
4,989032,4
5,659872,7
6,47159,4
7,11829,3
8,257869,6
9,465511,4


In [ ]:
z[['ID','E_VALORMATRICULAUNIVERSIDAD',]].head(10)

,ID,E_VALORMATRICULAUNIVERSIDAD
0,904256,Entre 5.5 millones y menos de 7 millones
1,645256,Entre 2.5 millones y menos de 4 millones
2,308367,Entre 2.5 millones y menos de 4 millones
3,470353,Entre 4 millones y menos de 5.5 millones
4,989032,Entre 2.5 millones y menos de 4 millones
5,659872,Más de 7 millones
6,47159,Entre 2.5 millones y menos de 4 millones
7,11829,Entre 1 millón y menos de 2.5 millones
8,257869,Entre 5.5 millones y menos de 7 millones
9,465511,Entre 2.5 millones y menos de 4 millones


## **Columna: E_HORASSEMANATRABAJA**
Los rangos de horas tienen un orden claro. Se mapearán de 0 a 4.

In [ ]:
df_procesado.E_HORASSEMANATRABAJA.unique()

array(['Menos de 10 horas', '0', 'Más de 30 horas', 'Entre 21 y 30 horas',
       'Entre 11 y 20 horas'], dtype=object)

In [ ]:
mapa_horas = {
    '0': 0, 'Menos de 10 horas': 1, 'Entre 11 y 20 horas': 2,
    'Entre 21 y 30 horas': 3, 'Más de 30 horas': 4
}
df_procesado['E_HORASSEMANATRABAJA'] = df_procesado['E_HORASSEMANATRABAJA'].map(mapa_horas)

In [ ]:
df_procesado.E_HORASSEMANATRABAJA.unique()

array([1, 0, 4, 3, 2])

In [ ]:
df_procesado[['ID','E_HORASSEMANATRABAJA',]].head(10)

,ID,E_HORASSEMANATRABAJA
0,904256,1
1,645256,0
2,308367,4
3,470353,0
4,989032,3
5,659872,1
6,47159,3
7,11829,2
8,257869,1
9,465511,4


In [ ]:
z[['ID','E_HORASSEMANATRABAJA',]].head(10)

,ID,E_HORASSEMANATRABAJA
0,904256,Menos de 10 horas
1,645256,0
2,308367,Más de 30 horas
3,470353,0
4,989032,Entre 21 y 30 horas
5,659872,Menos de 10 horas
6,47159,Entre 21 y 30 horas
7,11829,Entre 11 y 20 horas
8,257869,Menos de 10 horas
9,465511,Más de 30 horas


## **Columna: F_ESTRATOVIVIENDA**
Los estratos tienen un orden numérico claro. El caso especial 'Sin Estrato' se mapeará a 0.

In [ ]:
df_procesado.F_ESTRATOVIVIENDA.unique()

array(['Estrato 3', 'Estrato 4', 'Estrato 5', 'Estrato 2', 'Estrato 1',
       'Estrato 6', 'Sin Estrato'], dtype=object)

In [ ]:
# Primero, manejamos el caso especial 'Sin Estrato'
df_procesado['F_ESTRATOVIVIENDA'] = df_procesado['F_ESTRATOVIVIENDA'].replace('Sin Estrato', '0')
# Ahora extraemos el número.
df_procesado['F_ESTRATOVIVIENDA'] = df_procesado['F_ESTRATOVIVIENDA'].str.replace('Estrato ', '').astype(int)

In [ ]:
df_procesado.F_ESTRATOVIVIENDA.unique()

array([3, 4, 5, 2, 1, 6, 0])

In [ ]:
df_procesado[['ID','F_ESTRATOVIVIENDA',]].head(10)

,ID,F_ESTRATOVIVIENDA
0,904256,3
1,645256,3
2,308367,3
3,470353,4
4,989032,3
5,659872,5
6,47159,2
7,11829,2
8,257869,1
9,465511,5


In [ ]:
z[['ID','F_ESTRATOVIVIENDA',]].head(10)

,ID,F_ESTRATOVIVIENDA
0,904256,Estrato 3
1,645256,Estrato 3
2,308367,Estrato 3
3,470353,Estrato 4
4,989032,Estrato 3
5,659872,Estrato 5
6,47159,Estrato 2
7,11829,Estrato 2
8,257869,Estrato 1
9,465511,Estrato 5


## **Columna: F_EDUCACIONPADRE y F_EDUCACIONMADRE**
 Los niveles educativos tienen una jerarquía. Usaremos el mismo mapa para ambas columnas, `asignando a 'No sabe' y 'No aplica' el mismo valor que 'Ninguno'`

In [ ]:
df_procesado['F_EDUCACIONPADRE'] = df_procesado['F_EDUCACIONPADRE'].str.replace(' ', '_')
sorted(df_procesado.F_EDUCACIONPADRE.unique())

['Educación_profesional_completa',
 'Educación_profesional_incompleta',
 'Ninguno',
 'No_Aplica',
 'No_sabe',
 'Postgrado',
 'Primaria_completa',
 'Primaria_incompleta',
 'Secundaria_(Bachillerato)_completa',
 'Secundaria_(Bachillerato)_incompleta',
 'Técnica_o_tecnológica_completa',
 'Técnica_o_tecnológica_incompleta']

In [ ]:
df_procesado['F_EDUCACIONMADRE'] = df_procesado['F_EDUCACIONMADRE'].str.replace(' ', '_')
sorted(df_procesado.F_EDUCACIONMADRE.unique())

['Educación_profesional_completa',
 'Educación_profesional_incompleta',
 'Ninguno',
 'No_Aplica',
 'No_sabe',
 'Postgrado',
 'Primaria_completa',
 'Primaria_incompleta',
 'Secundaria_(Bachillerato)_completa',
 'Secundaria_(Bachillerato)_incompleta',
 'Técnica_o_tecnológica_completa',
 'Técnica_o_tecnológica_incompleta']

In [ ]:
mapa_educacion = {
    'Ninguno': 0, 'No_Aplica': 0,'No_sabe': 0, 'Primaria_incompleta': 1, 'Primaria_completa': 2,
    'Secundaria_(Bachillerato)_incompleta': 3, 'Secundaria_(Bachillerato)_completa': 4,
    'Técnica_o_tecnológica_incompleta': 5, 'Técnica_o_tecnológica_completa': 6,
    'Educación_profesional_incompleta': 7, 'Educación_profesional_completa': 8,
    'Postgrado': 9
}
df_procesado['F_EDUCACIONPADRE'] = df_procesado['F_EDUCACIONPADRE'].map(mapa_educacion)
df_procesado['F_EDUCACIONMADRE'] = df_procesado['F_EDUCACIONMADRE'].map(mapa_educacion)

In [ ]:
sorted(df_procesado.F_EDUCACIONMADRE.unique())

[np.int64(0),
 np.int64(1),
 np.int64(2),
 np.int64(3),
 np.int64(4),
 np.int64(5),
 np.int64(6),
 np.int64(7),
 np.int64(8),
 np.int64(9)]

In [ ]:
sorted(df_procesado.F_EDUCACIONPADRE.unique())

[np.int64(0),
 np.int64(1),
 np.int64(2),
 np.int64(3),
 np.int64(4),
 np.int64(5),
 np.int64(6),
 np.int64(7),
 np.int64(8),
 np.int64(9)]

In [ ]:
df_procesado[['ID','F_EDUCACIONPADRE','F_EDUCACIONMADRE']].head(10)

,ID,F_EDUCACIONPADRE,F_EDUCACIONMADRE
0,904256,5,9
1,645256,6,5
2,308367,4,4
3,470353,0,4
4,989032,2,2
5,659872,8,4
6,47159,7,6
7,11829,1,3
8,257869,4,7
9,465511,9,9


In [ ]:
z[['ID','F_EDUCACIONPADRE','F_EDUCACIONMADRE']].head(10)

,ID,F_EDUCACIONPADRE,F_EDUCACIONMADRE
0,904256,Técnica o tecnológica incompleta,Postgrado
1,645256,Técnica o tecnológica completa,Técnica o tecnológica incompleta
2,308367,Secundaria (Bachillerato) completa,Secundaria (Bachillerato) completa
3,470353,No sabe,Secundaria (Bachillerato) completa
4,989032,Primaria completa,Primaria completa
5,659872,Educación profesional completa,Secundaria (Bachillerato) completa
6,47159,Educación profesional incompleta,Técnica o tecnológica completa
7,11829,Primaria incompleta,Secundaria (Bachillerato) incompleta
8,257869,Secundaria (Bachillerato) completa,Educación profesional incompleta
9,465511,Postgrado,Postgrado


# **Columnas con Variables Nominales (Sin orden)**
Usaremos One-Hot Encoding.

## **Columna: E_PRGM_DEPARTAMENTO**
  Para evitar crear demasiadas columnas, solo codificaremos los 10 departamentos más comunes y agruparemos el resto en una categoría `"OTRO"`.

In [ ]:
top_10_dptos = df_procesado['E_PRGM_DEPARTAMENTO'].value_counts().nlargest(10).index
df_procesado['E_PRGM_DEPARTAMENTO'] = df_procesado['E_PRGM_DEPARTAMENTO'].where(df_procesado['E_PRGM_DEPARTAMENTO'].isin(top_10_dptos), 'OTRO')

dummies_depto = pd.get_dummies(df_procesado['E_PRGM_DEPARTAMENTO'], prefix='DEPTO')
df_procesado = pd.concat([df_procesado, dummies_depto], axis=1)
df_procesado = df_procesado.drop('E_PRGM_DEPARTAMENTO', axis=1)

In [ ]:
df_procesado.head()

,ID,E_PRGM_ACADEMICO,E_VALORMATRICULAUNIVERSIDAD,E_HORASSEMANATRABAJA,F_ESTRATOVIVIENDA,F_TIENEINTERNET,F_EDUCACIONPADRE,F_TIENELAVADORA,F_TIENEAUTOMOVIL,E_PAGOMATRICULAPROPIO,...,DEPTO_ATLANTICO,DEPTO_BOGOTÁ,DEPTO_BOLIVAR,DEPTO_BOYACA,DEPTO_CUNDINAMARCA,DEPTO_NARIÑO,DEPTO_NORTE SANTANDER,DEPTO_OTRO,DEPTO_SANTANDER,DEPTO_VALLE
0,904256,ENFERMERIA,6,1,3,1,5,1,1,0,...,False,True,False,False,False,False,False,False,False,False
1,645256,DERECHO,4,0,3,0,6,1,0,0,...,True,False,False,False,False,False,False,False,False,False
2,308367,MERCADEO Y PUBLICIDAD,4,4,3,1,4,1,0,0,...,False,True,False,False,False,False,False,False,False,False
3,470353,ADMINISTRACION DE EMPRESAS,5,0,4,1,0,1,0,0,...,False,False,False,False,False,False,False,False,True,False
4,989032,PSICOLOGIA,4,3,3,1,2,1,1,0,...,False,False,False,False,False,False,False,False,False,False


## **Columna: E_PRGM_ACADEMICO**
  Para evitar crear demasiadas columnas, solo codificaremos los 20 programas más comunes y agruparemos el resto en una categoría `"OTRO"`.

In [ ]:
top_20_programas = df_procesado['E_PRGM_ACADEMICO'].value_counts().nlargest(20).index
df_procesado['E_PRGM_ACADEMICO'] = df_procesado['E_PRGM_ACADEMICO'].where(df_procesado['E_PRGM_ACADEMICO'].isin(top_20_programas), 'OTRO')

dummies_prgm = pd.get_dummies(df_procesado['E_PRGM_ACADEMICO'], prefix='PRGM')
df_procesado = pd.concat([df_procesado, dummies_prgm], axis=1)
df_procesado = df_procesado.drop('E_PRGM_ACADEMICO', axis=1)

In [ ]:
df_procesado.head()

,ID,E_VALORMATRICULAUNIVERSIDAD,E_HORASSEMANATRABAJA,F_ESTRATOVIVIENDA,F_TIENEINTERNET,F_EDUCACIONPADRE,F_TIENELAVADORA,F_TIENEAUTOMOVIL,E_PAGOMATRICULAPROPIO,F_TIENECOMPUTADOR,...,PRGM_INGENIERIA DE SISTEMAS,PRGM_INGENIERIA INDUSTRIAL,PRGM_INGENIERIA MECANICA,PRGM_LICENCIATURA EN PEDAGOGIA INFANTIL,PRGM_MEDICINA,PRGM_NEGOCIOS INTERNACIONALES,PRGM_OTRO,PRGM_PSICOLOGIA,PRGM_PSICOLOGÍA,PRGM_TRABAJO SOCIAL
0,904256,6,1,3,1,5,1,1,0,1,...,False,False,False,False,False,False,False,False,False,False
1,645256,4,0,3,0,6,1,0,0,1,...,False,False,False,False,False,False,False,False,False,False
2,308367,4,4,3,1,4,1,0,0,0,...,False,False,False,False,False,False,True,False,False,False
3,470353,5,0,4,1,0,1,0,0,1,...,False,False,False,False,False,False,False,False,False,False
4,989032,4,3,3,1,2,1,1,0,1,...,False,False,False,False,False,False,False,True,False,False


# **La variable Más Importante**
##### **Columna `RENDIMIENTO_GLOBAL`**
Una Variable Cualitativa Ordinal que es el objetivo a predecir. Para lo cual aplicaremos un mapeo.

In [ ]:
df_procesado.RENDIMIENTO_GLOBAL.unique()

array(['medio-alto', 'bajo', 'alto', 'medio-bajo'], dtype=object)

In [ ]:
mapa_rendimiento = {
    'bajo': 0,
    'medio-bajo': 1,
    'medio-alto': 2,
    'alto': 3
}
df_procesado['RENDIMIENTO_GLOBAL'] = df_procesado['RENDIMIENTO_GLOBAL'].map(mapa_rendimiento)

In [ ]:
df_procesado.RENDIMIENTO_GLOBAL.unique()

array([2, 0, 3, 1])

# DataFrame **PREPROCESADO**

In [ ]:
df_procesado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 692500 entries, 0 to 692499
Data columns (total 50 columns):
 #   Column                                    Non-Null Count   Dtype  
---  ------                                    --------------   -----  
 0   ID                                        692500 non-null  int64  
 1   E_VALORMATRICULAUNIVERSIDAD               692500 non-null  int64  
 2   E_HORASSEMANATRABAJA                      692500 non-null  int64  
 3   F_ESTRATOVIVIENDA                         692500 non-null  int64  
 4   F_TIENEINTERNET                           692500 non-null  int64  
 5   F_EDUCACIONPADRE                          692500 non-null  int64  
 6   F_TIENELAVADORA                           692500 non-null  int64  
 7   F_TIENEAUTOMOVIL                          692500 non-null  int64  
 8   E_PAGOMATRICULAPROPIO                     692500 non-null  int64  
 9   F_TIENECOMPUTADOR                         692500 non-null  int64  
 10  F_EDUCACIONMADRE    

In [ ]:
df_procesado.head()

,ID,E_VALORMATRICULAUNIVERSIDAD,E_HORASSEMANATRABAJA,F_ESTRATOVIVIENDA,F_TIENEINTERNET,F_EDUCACIONPADRE,F_TIENELAVADORA,F_TIENEAUTOMOVIL,E_PAGOMATRICULAPROPIO,F_TIENECOMPUTADOR,...,PRGM_INGENIERIA DE SISTEMAS,PRGM_INGENIERIA INDUSTRIAL,PRGM_INGENIERIA MECANICA,PRGM_LICENCIATURA EN PEDAGOGIA INFANTIL,PRGM_MEDICINA,PRGM_NEGOCIOS INTERNACIONALES,PRGM_OTRO,PRGM_PSICOLOGIA,PRGM_PSICOLOGÍA,PRGM_TRABAJO SOCIAL
0,904256,6,1,3,1,5,1,1,0,1,...,False,False,False,False,False,False,False,False,False,False
1,645256,4,0,3,0,6,1,0,0,1,...,False,False,False,False,False,False,False,False,False,False
2,308367,4,4,3,1,4,1,0,0,0,...,False,False,False,False,False,False,True,False,False,False
3,470353,5,0,4,1,0,1,0,0,1,...,False,False,False,False,False,False,False,False,False,False
4,989032,4,3,3,1,2,1,1,0,1,...,False,False,False,False,False,False,False,True,False,False
